In [ ]:
import requests
import boto3

# Use API key to obtain a temporary access token (for authentication & authorization with Curation API routes)

api_key = open("<path_to_file_containing_api_key>").read().strip()  # substitute path to file containing api key
access_token_headers = {"x-api-key": api_key}
access_token_url = "https://api.cellxgene.dev.single-cell.czi.technology/curation/v1/auth/token"
resp = requests.post(access_token_url, headers=access_token_headers)
access_token = resp.json().get("access_token")

print(access_token)

In [ ]:
# Use access token to obtain temporary s3 write credentials for a given Collection; these credentials will only work for THIS Collection.

collection_id = "<private/revision_collection_uuid>"  # substitute your (non-public) Collection id, e.g. "1234abcd-5678-efab-1a2b-3c4d5e6f8a9b"
s3_credentials_url = f"https://api.cellxgene.dev.single-cell.czi.technology/curation/v1/collections/{collection_id}/datasets/s3-upload-credentials"
s3_cred_headers = {"Authorization": f"Bearer {access_token}"}
resp = requests.post(s3_credentials_url, headers=s3_cred_headers)
s3_creds = resp.json().get("Credentials")

In [ ]:
# Upload file using temporary s3 credentials

access_key_id, secret_access_key, session_token = s3_creds["AccessKeyId"], s3_creds["SecretAccessKey"], s3_creds["SessionToken"]
session = boto3.Session(
    aws_session_token=session_token,
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
)

s3 = session.client("s3")
success = s3.upload_file(
    Filename="<path_to_your_local_h5ad_datafile>",  # Your dataset, e.g. "~/my_datasets/Dataset1.h5ad"
    Bucket="cellxgene-dataset-submissions-dev",
    Key=f"{collection_id}/<arbitrary/curator-provided_tag.h5ad>",  # **TAG MUST POSSESS '.h5ad' SUFFIX** -- this tag will uniquely identify this Dataset within this Collection, e.g. "Dataset1.h5ad"
)
if success:
    print("File successfully uploaded")
else:
    print("File upload failed")